<a href="https://colab.research.google.com/github/Sbu97/AI_in_built_environments_sbusiso_IAAC2024_tutorials_/blob/main/notebooks/02_missing_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<a href="https://colab.research.google.com/github/STASYA00/IAAC2024_tutorials/blob/main/notebooks/02_missing_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Data

⚠️ **NOTE:** Kaggle files have already been uploaded to the repo. You find them in folder `kaggle_data`´

If you for any reason need to download data from kaggle, instructions are in one of the [optional notebooks](./88_kaggle_data.ipynb) : <a href="https://colab.research.google.com/github/STASYA00/IAAC2024_tutorials/blob/main/notebooks/88_kaggle_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!git clone https://github.com/STASYA00/IAAC2024_tutorials
%cd IAAC2024_tutorials/notebooks

Cloning into 'IAAC2024_tutorials'...
remote: Enumerating objects: 133, done.
remote: Counting objects: 100% (44/44), done.
remote: Compressing objects: 100% (28/28), done.
remote: Total 133 (delta 23), reused 35 (delta 16), pack-reused 89
Receiving objects: 100% (133/133), 9.24 MiB | 17.23 MiB/s, done.
Resolving deltas: 100% (54/54), done.
/content/IAAC2024_tutorials/notebooks


In [3]:
# importing the necessary packages

from matplotlib import pyplot as plt
import pandas as pd
import seaborn as sns


## 🏡 Buildings' Efficiency - Missing data

### The why of missing values

There are several reasons for having missing values, some of which are:
* equipment failure (e.g. GPS sensors), missing cells in manual data entry, censored/anonymous data
* unfilled optional values (e.g. "anything else you would like to add" - question in a survey)
* data was not originally collected (two datasets were merged, one had "year" records and another one didn't)
* and so on

For us it is important to understand why the values might be missing, as it gives us an understanding of how to treat them. There are several strategies:
* Remove rows or columns with missing values (used when there are many missing values along one axis)
* Assign an arbitrary value, like 18783 or 0 as to say "this value is missing" (used when we can not remove records from the db)
* Fill in missing values using some logic (e.g. assuming the number of occupants in a building based on similar buildings around or getting the year it was built in from another data source).
    

**How to find missing values:**

[Click>>](https://www.google.com/search?q=pandas+find+missing+values&oq=pandas+find+missing+values&gs_lcrp=EgZjaHJvbWUyBggAEEUYOTIGCAEQLhhA0gEINTg2N2owajGoAgCwAgA&sourceid=chrome&ie=UTF-8)

One of the methods is to check for each cell whether it is a NaN value and look at the values count that we are already familiar with:

In [4]:
metadata = pd.read_csv("../kaggle_data/tutorial_results/01_metadata.csv", index_col=0)
metadata.head()

,building_id,site_id,primaryspaceusage,sub_primaryspaceusage,sqm,timezone,electricity,hotwater,water,gas,industry,subindustry,heatingtype,yearbuilt,numberoffloors,occupants,rating
136,432,3,0,2,4703.0,1,0,0,0,0,0,2,11,NaN,5.0,NaN,1.0
137,416,3,0,2,499.3,1,0,1,0,0,0,2,7,NaN,4.0,NaN,0.0
138,436,3,0,2,9061.0,1,0,0,0,0,0,2,7,2005.0,10.0,NaN,2.0
139,441,3,0,3,7579.0,1,0,0,0,0,0,2,7,1913.0,5.0,NaN,0.0
140,442,3,0,3,5295.0,1,0,1,0,0,0,2,7,1953.0,6.0,NaN,1.0


In [5]:
metadata.isna()

,building_id,site_id,primaryspaceusage,sub_primaryspaceusage,sqm,timezone,electricity,hotwater,water,gas,industry,subindustry,heatingtype,yearbuilt,numberoffloors,occupants,rating
136,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False
137,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False
138,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
139,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
140,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1631,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
1632,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False
1633,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,True,False
1634,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False


In [6]:
for c in metadata.columns:
    if len(metadata[c].isna().unique())>1:
        print(metadata[c].isna().value_counts())
        print("\n")

False    199
True      52
Name: yearbuilt, dtype: int64


False    224
True      27
Name: numberoffloors, dtype: int64


True     138
False    113
Name: occupants, dtype: int64




### Dealing with missing values

**First questions to ask:**

* Why is the data missing (is there any logic in it)?
* What is the distribution of the missing data?

**Choose the strategy based on your findings:**

* Delete (rows or columns)
* Inserting a representative value (mean, median or the most likely to occur in these entries)
* Fill missing values manually 🙂
* Create a new column that would flag that the value is missing (e.g. Missing_built_year with 0 and 1 showing whether the values are missing) and use a special value in the original column. Increases the difficulty of the data mining process.
* Use data mining methods to estimate the missing value based on the rest of the dataset. Used when the feature is considered important and there is a substantial set of non-missing observations for this feature.

## Model Imputation methods

Let's go through different imputation methods. Note: these are not the best methods for each column, but rather a

### 🫧 Single value

Methods for getting `min`, `max`, `mean`, `median`:

`df[column_name].min()`

👍🏻

What could the advantages and disadvantages be when using this method with our data?

### 🫧 Nearest neighbours

[Algorithm in 94 seconds](https://www.youtube.com/watch?v=zeFt_JCA3b4)

In [7]:
from sklearn.neighbors import KNeighborsRegressor
model = KNeighborsRegressor(n_neighbors=5)
cols_to_drop = ["occupants", "numberoffloors", "yearbuilt"] #, "building_id"]
X = metadata.loc[metadata["occupants"].isna()==False].drop(columns=cols_to_drop)
Y = metadata.loc[metadata["occupants"].isna()==False]["occupants"]
missing_X = metadata.loc[metadata["occupants"].isna()].drop(columns=cols_to_drop)
model.fit(X, Y)

KNeighborsRegressor()

In [8]:
missing_Y = model.predict(missing_X)
missing_Y

array([ 195.8,   18.2, 1156.4,  745.6,  344.8,  359.6, 1019.8,  341.8,
       1156.4, 1082. , 1260.6,  336.6,  194.8, 1256.2,  832. ,  507. ,
       1160.8,  745.6,  344.8,  336.6,  184.4,  294.2,  359.6, 1043.8,
       1043.8,  344.8,  415.4,  745.6,  415.4,  195.8,  415.4,  334.6,
        415.4,  865.6,  344.8,  195.8,  164.6,  164.6,  173.6,  344.8,
         81.4,  390.6,  745.6,  966. ,  745.6,  415.4,  395.8,   99.6,
        195.8,  184.4,  371.4,  239.6,   18.2,  957.2,  425. ,   66.6,
        131. ,  193.2,   37.8,  407.8,   82.4,  344.8,   71.4,  195.8,
         82.4,   71.8,  425. ,  160.2,   66.6,  164.6,   71.8,   37.8,
        129.8,  365.6, 1160.8,  216.4,   53.8,  193.2,  205.6,  365.6,
         54.6,  165.4,  129.8,  175. ,   18.2,  351.6,  129.8,   53.8,
         34.8,  376.2,  333.6,  182. ,  307.8,  397.6,  195.8, 1019.8,
         18.2,   71. ,  351.6,   23.4,   59.4,  348.8, 1214.8,   23.4,
        357.4,  359.6, 1160.8,  359.4,  884.4, 1050.2, 1050.2,  857.6,
      

Thinking about how the algorithm works, do you think some columns are more important than the others? Are there any columns that are not important?

In [9]:
model = KNeighborsRegressor(n_neighbors=5)
cols_to_drop = ["occupants", "numberoffloors", "yearbuilt", "building_id"]
X = metadata.loc[metadata["occupants"].isna()==False].drop(columns=cols_to_drop)
Y = metadata.loc[metadata["occupants"].isna()==False]["occupants"]
missing_X = metadata.loc[metadata["occupants"].isna()].drop(columns=cols_to_drop)
model.fit(X, Y)
missing_Y = model.predict(missing_X)
missing_Y

array([ 195.8,   18.2, 1156.4,  756.2,  344.8,  385.6, 1019.8,  397.6,
       1156.4, 1082. , 1260.6,  364.4,  122.2, 1256.2,  832. ,  507. ,
       1160.8,  745.6,  344.8,  364.4,  193.6,  331.4,  366.6, 1043.8,
       1043.8,  344.8,  415.4,  745.6,  415.4,  195.8,  415.4,  334.6,
        415.4, 1019.8,  344.8,  195.8,  164.6,  164.6,  193.2,  344.8,
        142. ,  397.6,  745.6,  966. ,  745.6,  415.4,  363. ,  113.2,
        195.8,  130. ,  346. ,  244.4,   18.2,  957.2,  341.8,   41.6,
        161.4,  193.2,   18.2,  385.6,   81.4,  344.8,   41.6,  195.8,
        106.4,   66.2,  341.8,  122.2,   41.6,  164.6,   85.8,   18.2,
        113.2,  364.4, 1160.8,  193.6,   41.6,  193.2,  295.6,  346. ,
         71.4,   81.4,  113.2,   70. ,   18.2,  351.6,  169. ,   41.6,
         18.2,  363. ,  301.2,  146.2,  378.2,  397.6,  195.8, 1019.8,
         18.2,  142. ,  334.6,   66.2,   85.8,  377.6, 1214.8,   71.4,
        392. ,  385.6, 1160.8,  430.6,  884.4, 1050.2, 1050.2,  857.6,
      

#### Data scaling

![data scaling illustration](https://miro.medium.com/v2/resize:fit:1400/1*yR54MSI1jjnf2QeGtt57PA.png)

In [10]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)
missing_X_scaled = scaler.fit_transform(missing_X)

model.fit(X_scaled, Y)
missing_Y = model.predict(missing_X_scaled)
missing_Y

array([ 306.8,  306.8,  451. ,  379.2,  306.8,  306.8,  542.4,  216. ,
        451. ,  556.8,  542.4,  306.8,  216. ,  470.6,  451. ,  306.8,
        522.2,  306.8,  306.8,  344.8,  306.8,  306.8,  432. ,  485. ,
        485. ,  485. ,  485. ,  485. ,  489. ,  379.2,  485. ,  432. ,
        379.2,  485. ,  485. ,  485. ,  485. ,  489. ,  355. ,  489. ,
        379.2,  485. ,  485. ,  485. ,  485. ,  485. ,  485. ,  379.2,
        485. ,  379.2,  485. ,  485. ,   69.8,  853. ,  116.6,  159.8,
         64.4,   56.6,  122.4,  135.8,   64.4,  120.4,   29.2,  135.8,
         29.2,   29.2,  116.6,   65.2,  122.4,  148.6,  299.2,  159.8,
        210.8,  340.2, 1275.8,  265. ,  226. ,  393.2,   68.8,  299.2,
         88.2,   88.2,   88.2,   88.2,   88.2,  384.4,  384.4,  377.4,
        295. ,  339.2,  330.6,  299.2,  294. ,  299.2,  289.2,  397. ,
        119.6,  316.6,  306.8,  122.4,   82. ,  340.2,  542.4,  312.2,
        306.8,  306.8, 1465.6,  340.2, 1271.2,  485. ,  561.4,  397. ,
      

Have you noticed anything that doesn't turn with the predicted values?

In [11]:
missing_Y = missing_Y.astype(int)

#### Assigning generated values to the column

In [12]:
metadata["occupants"].loc[metadata["occupants"].isna()] = missing_Y

<ipython-input-12-7d3f308bea34>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata["occupants"].loc[metadata["occupants"].isna()] = missing_Y


### 🫧 Decision Tree

![decision tree illustration](https://cdn-images-1.medium.com/max/778/1*OLJWt9hD2zTS-x3gRhYuqg.png)

In [13]:
from sklearn.tree import DecisionTreeRegressor

cols_to_drop = ["numberoffloors", "yearbuilt"]

X = metadata.loc[metadata["yearbuilt"].isna()==False].drop(columns=cols_to_drop)
Y = metadata.loc[metadata["yearbuilt"].isna()==False]["yearbuilt"]
missing_X = metadata.loc[metadata["yearbuilt"].isna()].drop(columns=cols_to_drop)

tree = DecisionTreeRegressor()
tree.fit(X, Y)

DecisionTreeRegressor()

In [14]:
zipped = [(x, i) for i,x in sorted(zip(tree.feature_importances_, X), reverse=True)]
for f, v in zipped:
    print("{:18s}\t\t{:.2f}".format(f, v))

building_id       		0.37
sqm               		0.26
occupants         		0.13
rating            		0.09
heatingtype       		0.05
sub_primaryspaceusage		0.04
industry          		0.03
primaryspaceusage 		0.01
subindustry       		0.01
hotwater          		0.01
site_id           		0.01
gas               		0.00
water             		0.00
timezone          		0.00
electricity       		0.00


✨ **for the curious ones:**
* [zip function ](https://www.programiz.com/python-programming/methods/built-in/zip)
* [list comprehension](https://www.w3schools.com/python/python_lists_comprehension.asp)
* [enumerate](https://www.geeksforgeeks.org/enumerate-in-python/)

In [15]:
cols_to_drop = ["numberoffloors", "yearbuilt", "building_id"]

X = metadata.loc[metadata["yearbuilt"].isna()==False].drop(columns=cols_to_drop)
Y = metadata.loc[metadata["yearbuilt"].isna()==False]["yearbuilt"]
missing_X = metadata.loc[metadata["yearbuilt"].isna()].drop(columns=cols_to_drop)

tree = DecisionTreeRegressor()
tree.fit(X, Y)

DecisionTreeRegressor()

In [16]:
tree.predict(missing_X)

array([1906., 1995., 2006., 1906., 1900., 1945., 1919., 1919., 1976.,
       1966., 1919., 1976., 2000., 1906., 1902., 1906., 1976., 1919.,
       1976., 1906., 1906., 1906., 1960., 1970., 1976., 1976., 1906.,
       1976., 1906., 1960., 1906., 1906., 1976., 1906., 1976., 1976.,
       1906., 1906., 1906., 1906., 1906., 1906., 1906., 1976., 1906.,
       1906., 1906., 1976., 1906., 1906., 1906., 2004.])

#### 🧩 Task I - Assign predictions

In [17]:
metadata["yearbuilt"].loc[metadata["yearbuilt"].isna()] = tree.predict(missing_X).astype(int)

<ipython-input-17-a45f04d49cc5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata["yearbuilt"].loc[metadata["yearbuilt"].isna()] = tree.predict(missing_X).astype(int)


In [18]:
# check
# should return an empty dataframe, as we are not supposed to have nan values anymore
metadata["yearbuilt"].loc[metadata["yearbuilt"].isna()]

Series([], Name: yearbuilt, dtype: float64)

#### ⚙️ Solution

In [ ]:
metadata["yearbuilt"].loc[metadata["yearbuilt"].isna()] = tree.predict(missing_X).astype(int)

/tmp/ipykernel_28235/2202934034.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  metadata["yearbuilt"].loc[metadata["yearbuilt"].isna()] = tree.predict(missing_X).astype(int)


### 🎢 Saving our work

In [ ]:
# full_df.to_csv("../../kaggle_data/tutorial_results/02.csv")
metadata.to_csv("../kaggle_data/tutorial_results/02_metadata.csv")